In [11]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date

In [2]:
API_KEY = os.environ['BINANCE_API_KEY']
API_SECRET = os.environ['BINANCE_API_SECRET']

In [20]:

def get_klines(symbol, interval, start_time, end_time):
    url = 'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
    }
    headers = {
        'X-MBX-APIKEY': API_KEY
    }
    response = requests.get(url, params=params, headers=headers)
    return response.json()


def normalize(data):
    """Normalize the given data to the range [0, 1]."""
    min_val = min(data)
    max_val = max(data)
    return [(x - min_val) / (max_val - min_val) for x in data]

def create_plot(data, symbol):
    df = pd.DataFrame(data, columns=['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
                                      'Close Time', 'Quote Asset Volume', 'Number of Trades',
                                      'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume',
                                      'Ignore'])

    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    df['Close'] = df['Close'].astype(float)

    # Normalize the closing prices
    normalized_close = normalize(df['Close'].tolist())

    # Create directories for saving plots
    date_str = date.today() - timedelta(days = 1)
    actual_dir_path = f"../data/actual/{date_str}/{symbol}"
    normalized_dir_path = f"../data/normalized/{date_str}/{symbol}"
    os.makedirs(actual_dir_path, exist_ok=True)
    os.makedirs(normalized_dir_path, exist_ok=True)

    # Plot actual prices
    plt.figure(figsize=(10, 5))
    plt.plot(df['Open Time'], df['Close'], label='Actual Close Price', color='blue')
    plt.title(f'Actual Price Chart for {symbol}')
    plt.xlabel('Time')
    plt.ylabel('Price (USDT)')
    plt.legend()
    plt.grid()
    plt.xticks(rotation=45)
    plt.savefig(f'{actual_dir_path}/{symbol}_{date_str}.png')
    plt.close()

    # Plot normalized prices
    plt.figure(figsize=(10, 5))
    plt.plot(df['Open Time'], normalized_close, label='Normalized Close Price', color='green')
    plt.title(f'Normalized Price Chart for {symbol}')
    plt.xlabel('Time')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.grid()
    plt.xticks(rotation=45)
    plt.savefig(f'{normalized_dir_path}/{symbol}_{date_str}.png')
    plt.close()


In [21]:
def main():
    exchange_pairs = ["BTCUSDT", "ETHUSDT", "LTCUSDT"]
    interval = '15m'
    
    # Get start and end time for the previous day
    today = datetime.now()
    start_time = int((today - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0).timestamp() * 1000)
    end_time = int((today - timedelta(days=1)).replace(hour=23, minute=59, second=59, microsecond=999999).timestamp() * 1000)

    for pair in exchange_pairs:
        print(f'Fetching data for {pair}...')
        data = get_klines(pair, interval, start_time, end_time)
        if data:
            create_plot(data, pair)
        else:
            print(f'No data found for {pair}.')

if __name__ == '__main__':
    main()

Fetching data for BTCUSDT...
Fetching data for ETHUSDT...
Fetching data for LTCUSDT...
